In [12]:
'''Use multiple rounds to get a more robust results'''
import torch
import numpy as np
import pandas as pd
#import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc, balanced_accuracy_score
#import matplotlib.pyplot as plt
import torchvision
import shutil
import os

# DDI

In [13]:
def cal_metrics(csv_path, type_indices, is_binary=False, num=2):
    '''
    calculate average accuracy, accuracy per skin type, PQD, DPM, EOM.
    All known skin types
    input val results csv path, type_indices: a list
    output a dic, 'acc_avg': value, 'acc_per_type': array[x,x,x], 'PQD', 'DPM', 'EOM'
    '''
    df = pd.read_csv(csv_path)
    labels_array = np.zeros((3, len(df['label'].unique())))
    correct_array = np.zeros((3, len(df['label'].unique())))
    predictions_array = np.zeros((3, len(df['label'].unique())))
    positive_list = []  # get positive probability for binary classification
    
    for i in range(df.shape[0]):
        prediction = df.iloc[i]['prediction']
        label = df.iloc[i]['label']
        type = (df.iloc[i]['fitzpatrick']-1)//num
        labels_array[int(type), int(label)] += 1
        predictions_array[int(type),int(prediction)] += 1
        if prediction == label:
            correct_array[int(type), int(label)] += 1

        if is_binary:
            if prediction == 0:
                positive_list.append(1.0-df.iloc[i]['prediction_probability'])
            else:
                positive_list.append(df.iloc[i]['prediction_probability'])
    
    correct_array = correct_array[type_indices]
    labels_array = labels_array[type_indices]
    predictions_array = predictions_array[type_indices]

    # avg acc, acc per type
    correct_array_sumc, labels_array_sumc = np.sum(correct_array, axis=1), np.sum(labels_array, axis=1)  # sum skin conditions
    acc_array = correct_array_sumc/labels_array_sumc
    avg_acc = np.sum(correct_array)/np.sum(labels_array)

    # PQD
    PQD = acc_array.min()/acc_array.max()

    # DPM
    demo_array = predictions_array/np.sum(predictions_array, axis=1, keepdims=True)
    DPM = np.mean(demo_array.min(axis=0)/demo_array.max(axis=0))

    # EOM
    eo_array = correct_array/labels_array
    EOM = np.mean(np.min(eo_array,axis=0)/np.max(eo_array, axis=0))

    # if is binary classification, output AUC
    if is_binary:
        fpr, tpr, threshold = roc_curve(df['label'], positive_list,drop_intermediate=True)
        AUC = auc(fpr, tpr)
    else:
        AUC = -1

    return {'acc_avg': avg_acc, 'acc_per_type': acc_array, 'PQD': PQD, 'DPM': DPM, 'EOM': EOM, 'AUC': AUC}

In [15]:
# A56
epoch = 20
label = 'high'
holdout_set = 'random_holdout' # dermaamin br
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12_BaseBert'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34_BaseBert'
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a56_BaseBert'
model_name = 'fairdisco_ddireruns_15'
type_indices = [0,1,2]
csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False, num=[1,2][holdout_set=='random_holdout'])
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

acc_avg array
[0.83333333]
acc per type
[[0.88636364 0.7173913  0.9047619 ]]
PQD
[0.79290618]
DPM
[0.82674572]
EOM
[0.66983752]
AUC
[-1.]


In [17]:
# A56
epoch = 20
label = 'high'
holdout_set = 'a12' # dermaamin br
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12_BaseBert'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34_BaseBert'
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a56_BaseBert'
model_name = 'patchgotp_ddi_outdomain_a12_3_4'
type_indices = [1,2]
csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False, num=[2,1][holdout_set=='random_holdout'])
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

acc_avg array
[0.73883929]
acc per type
[[0.73858921 0.73913043]]
PQD
[0.99926776]
DPM
[0.86756156]
EOM
[0.85770962]
AUC
[-1.]


In [15]:
# A56
epoch = 20
label = 'high'
holdout_set = 'a34' # dermaamin br
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12_BaseBert'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34_BaseBert'
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a56_BaseBert'
model_name = 'patchgotp_ddi_outdomain_a12_3_4'
type_indices = [0,2]
csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False, num=[2,1][holdout_set=='random_holdout'])
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

acc_avg array
[0.7686747]
acc per type
[[0.75480769 0.7826087 ]]
PQD
[0.9644765]
DPM
[0.81764367]
EOM
[0.88215996]
AUC
[-1.]


In [27]:
# A56
epoch = 20
label = 'high'
holdout_set = 'a56' # dermaamin br
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12_BaseBert'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34_BaseBert'
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a56_BaseBert'
model_name = 'patchgotp_ddi_outdomain_a56_28736_1237892'
type_indices = [0,1]
csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False, num=[2,1][holdout_set=='random_holdout'])
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)
print(f'{avg_array[0]},{acc_per_type_array[0][0]},{acc_per_type_array[0][1]},{PQD_array[0]},{DPM_array[0]},{EOM_array[0]}')

acc_avg array
[0.72605791]
acc per type
[[0.76923077 0.68879668]]
PQD
[0.89543568]
DPM
[0.60785729]
EOM
[0.86856898]
AUC
[-1.]
0.7260579064587973,0.7692307692307693,0.6887966804979253,0.8954356846473028,0.6078572863132217,0.8685689844800195


# FITZPATRICK

In [56]:
def cal_metrics(csv_path, type_indices, is_binary=False):
    '''
    calculate average accuracy, accuracy per skin type, PQD, DPM, EOM.
    All known skin types
    input val results csv path, type_indices: a list
    output a dic, 'acc_avg': value, 'acc_per_type': array[x,x,x], 'PQD', 'DPM', 'EOM'
    '''
    df = pd.read_csv(csv_path)
    labels_array = np.zeros((6, len(df['label'].unique())))
    correct_array = np.zeros((6, len(df['label'].unique())))
    predictions_array = np.zeros((6, len(df['label'].unique())))
    positive_list = []  # get positive probability for binary classification
    for i in range(df.shape[0]):
        prediction = df.iloc[i]['prediction']
        label = df.iloc[i]['label']
        type = df.iloc[i]['fitzpatrick']-1
        labels_array[int(type), int(label)] += 1
        predictions_array[int(type),int(prediction)] += 1
        if prediction == label:
            correct_array[int(type), int(label)] += 1

        if is_binary:
            if prediction == 0:
                positive_list.append(1.0-df.iloc[i]['prediction_probability'])
            else:
                positive_list.append(df.iloc[i]['prediction_probability'])
    
    correct_array = correct_array[type_indices]
    labels_array = labels_array[type_indices]
    predictions_array = predictions_array[type_indices]

    # avg acc, acc per type
    correct_array_sumc, labels_array_sumc = np.sum(correct_array, axis=1), np.sum(labels_array, axis=1)  # sum skin conditions
    acc_array = correct_array_sumc/labels_array_sumc
    avg_acc = np.sum(correct_array)/np.sum(labels_array)

    # PQD
    PQD = acc_array.min()/acc_array.max()

    # DPM
    demo_array = predictions_array/np.sum(predictions_array, axis=1, keepdims=True)
    DPM = np.mean(demo_array.min(axis=0)/demo_array.max(axis=0))

    # EOM
    eo_array = correct_array/labels_array
    EOM = np.mean(np.min(eo_array,axis=0)/np.max(eo_array, axis=0))

    # if is binary classification, output AUC
    if is_binary:
        fpr, tpr, threshold = roc_curve(df['label'], positive_list,drop_intermediate=True)
        AUC = auc(fpr, tpr)
    else:
        AUC = -1

    return {'acc_avg': avg_acc, 'acc_per_type': acc_array, 'PQD': PQD, 'DPM': DPM, 'EOM': EOM, 'AUC': AUC}

In [34]:
epoch = 20
label = 'high'
holdout_set = 'a56' # dermaamin br
model_name = 'patchmotp_fitz_a56_2_2'
type_indices = [0,1,2,3]
csv_folder_list = ['S36', 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False)
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)
print(f'{avg_array[0]},{acc_per_type_array[0][0]},{acc_per_type_array[0][1]},{PQD_array[0]},{DPM_array[0]},{EOM_array[0]}')

acc_avg array
[0.77568456 0.77568456 0.77568456 0.77568456 0.77568456]
acc per type
[[0.74422613 0.75558836 0.78214503 0.83537906]
 [0.74422613 0.75558836 0.78214503 0.83537906]
 [0.74422613 0.75558836 0.78214503 0.83537906]
 [0.74422613 0.75558836 0.78214503 0.83537906]
 [0.74422613 0.75558836 0.78214503 0.83537906]]
PQD
[0.89088435 0.89088435 0.89088435 0.89088435 0.89088435]
DPM
[0.74742518 0.74742518 0.74742518 0.74742518 0.74742518]
EOM
[0.78104168 0.78104168 0.78104168 0.78104168 0.78104168]
AUC
[-1. -1. -1. -1. -1.]
0.7756845564074479,0.7442261289210617,0.7555883593420498,0.8908843462019623,0.747425175389952,0.781041675181064


In [11]:
epoch = 20
label = 'high'
holdout_set = 'random_holdout' # dermaamin br
model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b200_step2_gamma0.8'
type_indices = [0, 1, 2, 3, 4, 5]
csv_folder_list = ['S36', 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False)
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

acc_avg array
[0.8504527 0.8504527 0.8504527 0.8504527 0.8504527]
acc per type
[[0.8361204  0.82448537 0.85610465 0.88091068 0.89644013 0.8245614 ]
 [0.8361204  0.82448537 0.85610465 0.88091068 0.89644013 0.8245614 ]
 [0.8361204  0.82448537 0.85610465 0.88091068 0.89644013 0.8245614 ]
 [0.8361204  0.82448537 0.85610465 0.88091068 0.89644013 0.8245614 ]
 [0.8361204  0.82448537 0.85610465 0.88091068 0.89644013 0.8245614 ]]
PQD
[0.91973278 0.91973278 0.91973278 0.91973278 0.91973278]
DPM
[0.51514403 0.51514403 0.51514403 0.51514403 0.51514403]
EOM
[0.70571672 0.70571672 0.70571672 0.70571672 0.70571672]
AUC
[-1. -1. -1. -1. -1.]


In [12]:
print('average accuracy mean: {}, std: {}'.format(avg_array.mean(), avg_array.std()))
print('accuracy per skin type mean and std')
print(np.mean(acc_per_type_array, axis=0), np.std(acc_per_type_array, axis=0))
print('PQD mean: {}, std: {}'.format(PQD_array.mean(), PQD_array.std()))
print('DPM mean: {}, std: {}'.format(DPM_array.mean(), DPM_array.std()))
print('EOM mean: {}, std: {}'.format(EOM_array.mean(), EOM_array.std()))
print('AUC mean: {}, std: {}'.format(AUC_array.mean(), AUC_array.std()))

average accuracy mean: 0.8504527005931939, std: 0.0
accuracy per skin type mean and std
[0.8361204  0.82448537 0.85610465 0.88091068 0.89644013 0.8245614 ] [1.11022302e-16 0.00000000e+00 1.11022302e-16 1.11022302e-16
 0.00000000e+00 0.00000000e+00]
PQD mean: 0.9197327815825806, std: 0.0
DPM mean: 0.5151440327470146, std: 0.0
EOM mean: 0.7057167153639364, std: 0.0
AUC mean: -1.0, std: 0.0


In [13]:
epoch = 20
label = 'high'
holdout_set = 'random_holdout' # dermaamin br
model_name = 'BASE'
type_indices = [0, 1, 2]
csv_folder_list = ['S36', 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    csv_path = "training_BASE_20_high_random_holdout.csv"
    dic = cal_metrics(csv_path, type_indices, is_binary=False)
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

KeyError: 'label'

# GHC

In [15]:
epoch = 20
label = 'high'
holdout_set = 'random_holdout' # dermaamin br
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
type_indices = [0,1, 2, 3, 4, 5]
csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False)
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

acc_avg array
[0.88167343]
acc per type
[[0.84448161 0.85807151 0.88372093 0.93169877 0.92880259 0.87719298]]
PQD
[0.90638909]
DPM
[0.70386418]
EOM
[0.76212554]
AUC
[-1.]


In [17]:
# A12
epoch = 20
label = 'high'
holdout_set = 'a12' # dermaamin br
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
type_indices = [ 2, 3, 4, 5]
csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False)
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

acc_avg array
[0.83940899]
acc per type
[[0.8452237  0.83854729 0.83039791 0.83464567]]
PQD
[0.98245933]
DPM
[0.64189896]
EOM
[0.69082996]
AUC
[-1.]


In [34]:
# A34
epoch = 20
label = 'high'
holdout_set = 'a12' # dermaamin br
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
model_name='best_model_01_beta_0.6'
model_name='best_model_a12'
type_indices = [ 2,3,4,5]
csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False)
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print([i for i in acc_per_type_array[0]])
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

FileNotFoundError: [Errno 2] No such file or directory: 'results_best_model_a12_20_high_a12.csv'

In [58]:
# A56
epoch = 20
label = 'high'
holdout_set = 'random_holdout' # dermaamin br
#model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
#model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
#model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34'
model_name='mgot_ddt_fd_run'
type_indices = [1,2,3]
csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
is_binary = True

avg_array = np.zeros((len(csv_folder_list)))
acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
PQD_array = np.zeros((len(csv_folder_list)))
DPM_array = np.zeros((len(csv_folder_list)))
EOM_array = np.zeros((len(csv_folder_list)))
AUC_array = np.zeros((len(csv_folder_list)))

for i in range(len(csv_folder_list)):
    csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
    #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
    dic = cal_metrics(csv_path, type_indices, is_binary=False)
    avg_array[i] = dic['acc_avg']
    acc_per_type_array[i, :] = dic['acc_per_type']
    PQD_array[i] = dic['PQD']
    DPM_array[i] = dic['DPM']
    EOM_array[i] = dic['EOM']
    AUC_array[i] = dic['AUC']


print('acc_avg array')
print(avg_array)
print('acc per type')
print(acc_per_type_array)
print('PQD')
print(PQD_array)
print('DPM')
print(DPM_array)
print('EOM')
print(EOM_array)
print('AUC')
print(AUC_array)

acc_avg array
[0.79545455]
acc per type
[[       nan 0.79545455        nan]]
PQD
[nan]
DPM
[nan]
EOM
[nan]
AUC
[-1.]


/tmp/ipykernel_561564/3204821965.py:34: RuntimeWarning: invalid value encountered in divide
  acc_array = correct_array_sumc/labels_array_sumc
/tmp/ipykernel_561564/3204821965.py:41: RuntimeWarning: invalid value encountered in divide
  demo_array = predictions_array/np.sum(predictions_array, axis=1, keepdims=True)
/tmp/ipykernel_561564/3204821965.py:45: RuntimeWarning: invalid value encountered in divide
  eo_array = correct_array/labels_array


In [5]:
# A56
for i in ['','_0','_1','_2','_3']:
    epoch = 20
    label = 'high'
    holdout_set = 'random_holdout' # dermaamin br
    #model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
    #model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
    #model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
    #model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
    #model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34'
    #model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12_BaseBert'
    model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34_BaseBert'
    #model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a56_BaseBert'
    model_name = f'dxenwskl{i}'
    print(model_name)
    type_indices = [0,1,2,3,4, 5]
    csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
    is_binary = True
    
    avg_array = np.zeros((len(csv_folder_list)))
    acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
    PQD_array = np.zeros((len(csv_folder_list)))
    DPM_array = np.zeros((len(csv_folder_list)))
    EOM_array = np.zeros((len(csv_folder_list)))
    AUC_array = np.zeros((len(csv_folder_list)))
    
    for i in range(len(csv_folder_list)):
        csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
        #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
        dic = cal_metrics(csv_path, type_indices, is_binary=False)
        avg_array[i] = dic['acc_avg']
        acc_per_type_array[i, :] = dic['acc_per_type']
        PQD_array[i] = dic['PQD']
        DPM_array[i] = dic['DPM']
        EOM_array[i] = dic['EOM']
        AUC_array[i] = dic['AUC']
    
    
    print('acc_avg array')
    print(avg_array)
    print('acc per type')
    print(acc_per_type_array)
    print('PQD')
    print(PQD_array)
    print('DPM')
    print(DPM_array)
    print('EOM')
    print(EOM_array)
    print('AUC')
    print(AUC_array)
    print(

dxenwskl
acc_avg array
[0.87980019]
acc per type
[[0.85031847 0.85995851 0.88822653 0.91821561 0.91958042 0.87931034]]
PQD
[0.92468092]
DPM
[0.7109759]
EOM
[0.77322705]
AUC
[-1.]
dxenwskl_0
acc_avg array
[0.871995]
acc per type
[[0.86833333 0.83519837 0.8816568  0.91746641 0.89152542 0.890625  ]]
PQD
[0.91033128]
DPM
[0.5384108]
EOM
[0.70767323]
AUC
[-1.]
dxenwskl_1
acc_avg array
[0.88011239]
acc per type
[[0.85785124 0.87130802 0.85905045 0.92128801 0.93203883 0.85185185]]
PQD
[0.91396605]
DPM
[0.45105208]
EOM
[0.51685558]
AUC
[-1.]
dxenwskl_2
acc_avg array
[0.87823915]
acc per type
[[0.83639399 0.86340469 0.88888889 0.90275229 0.93537415 0.89908257]]
PQD
[0.89418121]
DPM
[0.36074237]
EOM
[0.43630995]
AUC
[-1.]
dxenwskl_3
acc_avg array
[0.88604433]
acc per type
[[0.82646048 0.87991718 0.88041854 0.92486583 0.92976589 0.9609375 ]]
PQD
[0.86005644]
DPM
[0.44647448]
EOM
[0.82247225]
AUC
[-1.]


In [3]:
acc_avg array
[0.83940899]
acc per type
[[0.8452237  0.83854729 0.83039791 0.83464567]]
PQD
[0.98245933]
DPM
[0.64189896]
EOM
[0.69082996]
AUC
[-1.]

SyntaxError: invalid syntax (521987650.py, line 1)

In [15]:
# A56
for mi in range(5):
    epoch = 20
    label = 'high'
    holdout_set = 'random_holdout' # dermaamin br
    #model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_CompleteAlignment'
    #model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT"
    #model_name = "MOT_DisCo_lamb0.9_a0.5_got0.01_b64_step2_gamma0.8_BATCHWISEALIGNMENT_DONTCONSIDERTHIS_BIOBERT_Trial2"
    #model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12'
    #model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34'
    #model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a12_BaseBert'
    model_name='MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a34_BaseBert'
    #model_name = 'MOT_DisCo_lamb0.9_a0.5_got0.01_b32_step2_gamma0.8_BATCHWISEALIGNMENT_OUTCLASS_a56_BaseBert'
    model_name = f'marlsdk_{mi}'
    type_indices = [0,1,2,3,4, 5]
    csv_folder_list = ['S36']#, 'S37', 'S38', 'S39', 'S40']
    is_binary = True
    
    avg_array = np.zeros((len(csv_folder_list)))
    acc_per_type_array = np.zeros((len(csv_folder_list), len(type_indices)))
    PQD_array = np.zeros((len(csv_folder_list)))
    DPM_array = np.zeros((len(csv_folder_list)))
    EOM_array = np.zeros((len(csv_folder_list)))
    AUC_array = np.zeros((len(csv_folder_list)))
    
    for i in range(len(csv_folder_list)):
        csv_path = 'results_{}_{}_{}_{}.csv'.format(model_name, epoch,label,holdout_set)
        #csv_path = 'results/{}/results_{}_{}_{}_{}.csv'.format(csv_folder_list[i], model_name, epoch,label,holdout_set)
        dic = cal_metrics(csv_path, type_indices, is_binary=False)
        avg_array[i] = dic['acc_avg']
        acc_per_type_array[i, :] = dic['acc_per_type']
        PQD_array[i] = dic['PQD']
        DPM_array[i] = dic['DPM']
        EOM_array[i] = dic['EOM']
        AUC_array[i] = dic['AUC']
    
    
    # print('acc_avg array')
    # print(avg_array)
    # print('acc per type')
    # print(acc_per_type_array)
    # print('PQD')
    # print(PQD_array)
    # print('DPM')
    # print(DPM_array)
    # print('EOM')
    # print(EOM_array)
    # print('AUC')
    # print(AUC_array)
    
    print(avg_array[0],end=',')
    for i in acc_per_type_array[0]:
        print(i,end=',')
    print(PQD_array[0],end=',')
    print(DPM_array[0],end=',')
    print(EOM_array[0],end=',')
    print(AUC_array[0])

0.8804246019356853,0.8264604810996563,0.8592132505175983,0.874439461883408,0.9409660107334525,0.9331103678929766,0.9296875,0.8783106633739693,0.4662337260496405,0.6891720179449449,-1.0
0.8804246019356853,0.8264604810996563,0.8592132505175983,0.874439461883408,0.9409660107334525,0.9331103678929766,0.9296875,0.8783106633739693,0.4662337260496405,0.6891720179449449,-1.0
0.8810490165469872,0.8683333333333333,0.8504577822990844,0.878698224852071,0.9213051823416507,0.9186440677966101,0.9375,0.90715496778569,0.5256031099033817,0.715976089365424,-1.0
0.876053699656572,0.8181818181818182,0.8818565400843882,0.870919881305638,0.9194991055456172,0.9158576051779935,0.8425925925925926,0.889812522108242,0.4558232483196408,0.5460874616214423,-1.0
0.8751170777396191,0.8447412353923205,0.8583078491335372,0.8725925925925926,0.8990825688073395,0.9387755102040817,0.9174311926605505,0.8998330550918197,0.35270062672179403,0.5828831417780037,-1.0
